In [14]:
import pandas as pd

# قراءة البيانات
df = pd.read_csv('dataset.csv')

# تحويل عمود التاريخ إلى datetime
df['Date'] = pd.to_datetime(df['Date'], format='%b-%y')  # مثال: Jan-20

# إنشاء عمود الشهر والشهر-السنة
df['month'] = df['Date'].dt.to_period('M')
df['year'] = df['Date'].dt.to_period('Y')

# التأكد من النتيجة
print(df.head())


        Date      ?  ?.1  bills amount    month  year
0 2020-01-01  344.0  0.0         344.0  2020-01  2020
1 2020-02-01  344.0  0.0         344.0  2020-02  2020
2 2020-03-01  344.0  0.0         344.0  2020-03  2020
3 2020-04-01  344.0  0.0         344.0  2020-04  2020
4 2020-05-01  344.0  0.0         344.0  2020-05  2020


In [15]:
from prophet import Prophet

# إعادة تسمية الأعمدة لتوافق Prophet
df_prophet = df.rename(columns={'Date': 'ds', 'bills amount': 'y'})

# تأكد أن العمود y من نوع float
df_prophet['y'] = df_prophet['y'].astype(float)

print(df_prophet.head())


          ds      ?  ?.1      y    month  year
0 2020-01-01  344.0  0.0  344.0  2020-01  2020
1 2020-02-01  344.0  0.0  344.0  2020-02  2020
2 2020-03-01  344.0  0.0  344.0  2020-03  2020
3 2020-04-01  344.0  0.0  344.0  2020-04  2020
4 2020-05-01  344.0  0.0  344.0  2020-05  2020


In [16]:
model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
model.fit(df_prophet)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpwyecdqkx/235ktdm2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwyecdqkx/svbakmn6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=53392', 'data', 'file=/tmp/tmpwyecdqkx/235ktdm2.json', 'init=/tmp/tmpwyecdqkx/svbakmn6.json', 'output', 'file=/tmp/tmpwyecdqkx/prophet_modelj90y2l68/prophet_model-20251016104258.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
10:42:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:42:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [17]:
# هنا نتوقع لـ 12 شهر قادم
future = model.make_future_dataframe(periods=12, freq='MS')  # MS = Month Start
forecast = model.predict(future)

print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12))


           ds        yhat  yhat_lower  yhat_upper
70 2025-11-01  593.148942  490.829449  701.347317
71 2025-12-01  592.971953  487.017389  704.628795
72 2026-01-01  590.514747  486.369965  701.004146
73 2026-02-01  591.582897  492.027153  698.807298
74 2026-03-01  574.606912  470.343808  674.995411
75 2026-04-01  579.740688  476.136049  679.871200
76 2026-05-01  585.579631  483.882379  691.061918
77 2026-06-01  592.400626  489.267895  699.483306
78 2026-07-01  598.457349  490.150263  707.528541
79 2026-08-01  375.449197  267.807204  476.610597
80 2026-09-01  651.127206  547.010255  759.839079
81 2026-10-01  595.953564  496.167624  705.097597


In [18]:
from prophet.plot import plot_plotly
import plotly.offline as py

fig = plot_plotly(model, forecast)
py.plot(fig, filename='electricity_forecast.html')


'electricity_forecast.html'